# **CMPSC497 Fall 2022 Programming Assignment 2**
Task 2
Notebook by Anish Phule

# Task 1: Data Preprocessing

In [1]:
#Importing various libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import operator
from math import ceil 
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
#This makes all figures appear bigger in the notebook
plt.rcParams['figure.figsize'] = [10, 5]

In [3]:
#Reading the input csv file
df = pd.read_csv (r'/content/drive/MyDrive/CMPSC497/credit cards-2022-post.csv')
df.to_excel (r'/content/drive/MyDrive/CMPSC497/credit_cards.xlsx', index = None, header=True)

In [4]:
#Clearing unclear or questionable values
for index, row in df.iterrows():
    if (row['EDUCATION'] < 1):
        row['EDUCATION'] = 1
    elif (row['EDUCATION'] >4):
        row['EDUCATION'] = 4
    if (row['MARRIAGE'] < 1):
        row['MARRIAGE'] = 1
  
pay_list = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']
for index, row in df.iterrows():
    for i in range(len(pay_list)):
        if (row[pay_list[i]] < 0):
            row[pay_list[i]] = 0
        
df = df.reset_index()
df.to_excel (r'/content/drive/MyDrive/CMPSC497/credit_cards_cleared.xlsx', index = None, header=True)

##Task 1(a): Creating new Features

We create 6 new features, the Remaining balance, defined as 'Bill Amount - Pay Amount'. 

In [5]:
bill_amt = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']
pay_amt = ['PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']
#We create 6 new features called Remaining Balance, which is the difference between bill amount and pay amount
rem_bal = ['REM_BAL1','REM_BAL2','REM_BAL3','REM_BAL4','REM_BAL5','REM_BAL6']

for i in range(6):
    df[rem_bal[i]] = df[bill_amt[i]] - df[pay_amt[i]]

df.to_excel (r'/content/drive/MyDrive/CMPSC497/credit_cards_updated.xlsx', index = None, header=True)
print(df.head(3))

   index    ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  \
0      0  5771     180000    1          2         1   38      1      2      2   
1      1  3999      20000    2          1         2   23      2      2      2   
2      2   397     100000    1          1         2   38      0      0      0   

   ...  PAY_AMT4  PAY_AMT5  PAY_AMT6  default payment next month  REM_BAL1  \
0  ...       124         0         0                           1       826   
1  ...       500         0      1000                           1      6236   
2  ...         0       579         0                           0     38308   

   REM_BAL2  REM_BAL3  REM_BAL4  REM_BAL5  REM_BAL6  
0      4412      4071      4947      5195         0  
1     12559     11459     12457     13104     11744  
2     41453     41491     43011     42432     28947  

[3 rows x 32 columns]


## Task 1(b): Principle Component Analysis

We take the 18 features from Bill amount, Pay Amount and Remaining balance, and perform Principle Component Analysis to reduce the Dimensionality. 

In [6]:
#We first need to standardize our attributes for PCA
attr = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6','REM_BAL1','REM_BAL2','REM_BAL3','REM_BAL4','REM_BAL5','REM_BAL6']
df_pca = df[attr]

for i in attr:
  mean = df_pca[i].mean()
  std = df_pca[i].std()
  df_pca[i]=(df_pca[i]- mean)/std

print(df_pca.head(3))

   BILL_AMT1  BILL_AMT2  BILL_AMT3  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  \
0  -0.661082  -0.627831  -0.612095  -0.596497  -0.580089  -0.656619 -0.216708   
1  -0.566379  -0.512940  -0.498629  -0.473366  -0.449176  -0.441485 -0.127637   
2  -0.140501  -0.091370  -0.068644  -0.004104   0.045857  -0.167959 -0.186899   

   PAY_AMT2  PAY_AMT3  PAY_AMT4  PAY_AMT5  PAY_AMT6  REM_BAL1  REM_BAL2  \
0 -0.237318 -0.274405 -0.294510 -0.310511 -0.291435 -0.614494 -0.532564   
1 -0.237318 -0.245231 -0.270887 -0.310511 -0.235529 -0.539932 -0.420507   
2 -0.196406 -0.253399 -0.302300 -0.273426 -0.291435 -0.097910 -0.023087   

   REM_BAL3  REM_BAL4  REM_BAL5  REM_BAL6  
0 -0.543989 -0.523671 -0.500633 -0.563364  
1 -0.437804 -0.406328 -0.369513 -0.366943  
2 -0.006167  0.071075  0.116704 -0.079220  


In [7]:
X_train = df_pca

from sklearn.decomposition import PCA

pca = PCA(n_components=3, random_state=100)
X = pca.fit_transform(X_train)

In [8]:
df_new = pd.DataFrame(X, columns = ['A','B','C'])
print(df_new.head(3))

          A         B         C
0 -2.098848 -0.332509 -0.076862
1 -1.650113 -0.298452 -0.108871
2 -0.213877 -0.537090 -0.109664


##Task1(c): Normalization

Before using the attributes for clustering, we must first normalize them. Here I am using Min-Max normalization, which involves subtracting the minimum value of an attribute, anddividing it by the range of the attribute. This brings all attribute values in between [0,1].

In [9]:
columns = ['A','B','C']
for i in columns:
  df_new[i] = (df_new[i] - df_new[i].min())/(df_new[i].max() - df_new[i].min())

print(df_new.head(3))

          A         B         C
0  0.076566  0.035675  0.371407
1  0.090043  0.036034  0.370660
2  0.133176  0.033519  0.370642


#Task 2: (Your own) Implementation of the Agglomerative Hierarchical Clustering (MIN) algorithm

In this section, we implement our own hierarchal clustering using MIN criteria.

##Task2(a): Represent the data points in a cluster using a data structure

We must represent the data points in form of a data structure, that can effectively capture the hierarchal clustering of the data points.\
For this task, I am representing the points in the form of a nested list. I also create a parallel nested index list that houses the indexes of the points that are in a cluster together.\
In the beginning, since each point is a cluster, we will have one index in its individual list, and a list containing all these lists. We also have a separate list of lists with each list containing the attribute value for each point.\
 When two points are clustered together, the index of the second point will be added to the list of the first point, and the second point's list will be removed. This goes on until until we have n_cluster remaining lists, with each list containing indexes of points that are clustered according to our proximity matrix.

For example, for first 5 points, the index list would be [[1],[2],[3],[4],[5]]. If the points 2 and 3 are merged, the lists [2] and [3] will be merged too, and the list will now look like [[1],[2 , 3],[4],[5]]. This goes on until the no. of lists inside the list is equal to n_cluster.

Illustration:\
step 1: Individual clusters - [ [1], [2], [3], [4], [5] ]\
step 2: cluster 2 and 3 merged - [ [1], [2 , 3], [4], [5] ]\
step 3: cluster 1 and 4 merged - [ [1 , 4],[2 , 3], [5] ]\
step 4: cluster [2 , 3] and 5 merged - [ [1 , 4], [2 , 3, 5] ]\
step 5: remaining 2 clusters merged - [ 1 , 4 , 2 , 3 , 5 ]\
Clustering done.

And the beautiful thing of this structure is, the indexes are added in order of clustering. So, if we do intend to build a tree or a dendogram, we can backtrack the lists to travel from root to individual leaves. We might need to store the corresponding dissimilarities in a separate list so we know what should be the height in case of a dendogram, but it is completely doable.

If we do intend to backtrack the clustering in order to know what were the clusters at what stage, we can create an additional list that appends the current clustering list to it every time clustering is performed. Thus we have a full record of the hierarchal clustering.

For the purpose of code block representation, I will be using only the first 5 data points to show code and results for section 2(a)-2(d). The full merged code with 100 points and 5 clusters is in section 2(e).

In [10]:
#create a new dataframe of first 5 points
df2 = df_new.head(5)

# Create two empty lists, one each for the attribute values and index of data points
Row_list =[]
index_list = []
  
# Iterate over each row
for index, rows in df2.iterrows():
    # Create list for the current row
    my_list =[rows.A, rows.B, rows.C]
    my_index = [index]  

    # append the list to the final list
    Row_list.append(my_list)
    index_list.append(my_index)

##Task 2(b): Proximity matrix

We need to compute a proximity matrix for comparing the similarity/Dissimilarity of data points. For this project, we create a dissimilarity matrix in the form of a distance matrix. We calculate the total Euclidean distance between the attribute values of each point, and use it as a dissimilarity measure. Most dissimilar point will have the highest value in the matrix and vice versa.

For this project, We use the distance_matrix function from the scipy library. This library takes in the attributes list of lists as input(as both x and y), and p=2 for euclidean distance. It then calculates pairwaise distance for each list in the attributes list. The great thing about this library, is that the x and y can be multidimensional, and the function can still easily calculate the euclidean distance.

In [11]:
# import important libraries
import numpy as np
from scipy.spatial import distance_matrix

# compute the distance matrix
dist_mat = distance_matrix(Row_list, Row_list, p=2)

# display distance matrix
print("Distance Matrix:\n", dist_mat)

Distance Matrix:
 [[0.         0.01350205 0.05665639 0.13411989 0.30826433]
 [0.01350205 0.         0.04320683 0.12062529 0.29493524]
 [0.05665639 0.04320683 0.         0.07756498 0.2520509 ]
 [0.13411989 0.12062529 0.07756498 0.         0.17630309]
 [0.30826433 0.29493524 0.2520509  0.17630309 0.        ]]


##Task 2(c): Find two closest clusters based on the MIN/Single Link scheme

For the MIN/Single link scheme, the two closest clusters are merged. We find the minimum value in the distance matrix(other than 0), and find the indices of that value in the matrix. These indices represent the clusters that are closest to each other. 


In [12]:
#We first create a masked array that masks the 0.0 values, so we can concentrate on the non-zero ones
masked_mat = np.ma.masked_equal(dist_mat, 0.0, copy=False)
#We now find the minimum value in the matrix.
#Note that since the min value will be in two locations, two index arrays will be output. Hence the (a,b)
(a,b) = np.where(dist_mat == masked_mat.min())

#Let's see the location
print(a)

[0 1]


##Task 2(d): Merge two clusters into a new cluster and update the proximity matrix

We now need to merge the two clusters found in the previous section into one, and update the proximity matrix. For both this tasks, we need not change the attribute lists. All clusterings from here on can be handled just on the basis of the previously calculated distance matrix.

To merge the clusters, we simply add the lists corresponding to the indices obtained from the previous section. We then 'pop' the list for the latter index, since we don't need it anymore.

For updating the proximity matrix, We compare the row and column values of distance matrix whose one index is the index of the either of location indexes found in the previous part. Then, both the rows/columns is equated to the minimum of the compared values, and one of the row and one of the column is deleted, since they are both the same.

In [13]:
index_list[a[0]] = index_list[a[0]] + index_list[a[1]] #Merging the index list of indices previously obtained
index_list.pop(a[1]) #Popping the second list after it has been merged

print("The merged Clusters are:")
print(index_list) #The list of lists after merging

The merged Clusters are:
[[0, 1], [2], [3], [4]]


As we can see, the lists 0 and 1 have been merged into one cluster.
We now update the proximity matrix

In [14]:
#We now update the proximity matrix for indices 0 and 1, obtained previously.
i = 0
while i<len(index_list):
  #Compare the row-wise values for 0 and 1, and equate it to the minimum one
  dist_mat[a[0], i+1] = dist_mat[a[1], i+1] = min(dist_mat[a[0], i+1],dist_mat[a[1], i+1])
  #Compare the column-wise values for 0 and 1, and equate it to the minimum one
  dist_mat[i+1, a[0]] = dist_mat[i+1, a[1]] = min(dist_mat[i+1, a[0]],dist_mat[i+1, a[1]])
  i+=1

#Delete one column and row, since we now have two identical rows and two identical columns
dist_mat = np.delete(dist_mat, a[0], 0)
dist_mat = np.delete(dist_mat, a[0], 1)

print("Updated Proximity matrix:")
print(dist_mat)

Updated Proximity matrix:
[[0.         0.04320683 0.12062529 0.29493524]
 [0.04320683 0.         0.07756498 0.2520509 ]
 [0.12062529 0.07756498 0.         0.17630309]
 [0.29493524 0.2520509  0.17630309 0.        ]]


##Task 2(e): Implement your own version 

We now put together the above parts to create a complete Agglomerative Hierarchical Clustering (MIN) algorithm. 

In the beginning of the algorithm, we define the number of clusters we want and the number of points we want to cluster together. We also create a new column in the dataframe for assigning cluster labels after clustering is done.

If we do intend to backtrack the clustering in order to know what were the clusters at what stage, we can create an additional list that appends the current clustering list to it every time clustering is performed. Thus we have a full record of the hierarchal clustering.



In [15]:
n_clusters = 5 #No. of clusters
df2 = df_new.head(100) #No. of points to be clustered
df2['labels'] = "" #Extra column added in dataframe for clusters.

##Creating data structure for clustering - Task 2(a)

# Create an empty list
Row_list =[]
index_list = []
list_track = [] #adds the current cluster list to this incase we need to check clustering history

# Iterate over each row
for index, rows in df2.iterrows():
    # Create list for the current row
    my_list =[rows.A, rows.B, rows.C]
    my_index = [index]  

    # append the list to the final list
    Row_list.append(my_list)
    index_list.append(my_index)


print("Initial Clusters:\n", index_list)
#print(list_track)
print("\n")
##Calculating the Proximity Matrix - Task 2(b)

# import important libraries
import numpy as np
from scipy.spatial import distance_matrix

# compute the distance matrix
dist_mat = distance_matrix(Row_list, Row_list, p=2)

# display distance matrix
print("Initial Distance Matrix:\n", dist_mat)

##Finding the two closest clusters based on MIN scheme - Task 2(c)

masked_mat = np.ma.masked_equal(dist_mat, 0.0, copy=False)
(a,b) = np.where(dist_mat == masked_mat.min())

##Merging two clusters into new cluster and updating proximity matrix - Task 2(d)

index_list[a[0]] = index_list[a[0]] + index_list[a[1]]
index_list.pop(a[1])

list_track.append(index_list)

k = len(dist_mat)
while k>2:
    if len(index_list) == n_clusters:   #n_cluster criterion
      break
    i = 0
    while i<k-1:
      dist_mat[a[0], i+1] = min(dist_mat[a[0], i+1],dist_mat[a[1], i+1])
      dist_mat[a[1], i+1] = min(dist_mat[a[0], i+1],dist_mat[a[1], i+1])
      dist_mat[i+1, a[0]] = dist_mat[i+1, a[1]] = min(dist_mat[i+1, a[0]],dist_mat[i+1, a[1]])
      i+=1
      
    dist_mat = np.delete(dist_mat, a[0], 0)
    dist_mat = np.delete(dist_mat, a[0], 1)

    #We repeat the steps in task 2(b), 2(c) and 2(d)
    masked_mat = np.ma.masked_equal(dist_mat, 0.0, copy=False)
    (a,b) = np.where(dist_mat == masked_mat.min())
    index_list[a[0]] = index_list[a[0]] + index_list[a[1]]
    index_list.pop(a[1])
    #list_track.append(index_list)
    k-=1
print("\n")
print("Done\n")
print("Final Proximity matrix")
print(dist_mat)
print("\n")
print("The clusters are(list of lists):")
[print(c) for c in index_list]
print("\n")


#Assigning labels to each data point based on their clustering
for count in range(n_clusters):
  for index in index_list[count]:
    df2['labels'][index] = count+1
print("First 10 data points with the attributes and labels:\n")
print(df2.head(10))

Initial Clusters:
 [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [83], [84], [85], [86], [87], [88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99]]


Initial Distance Matrix:
 [[0.         0.01350205 0.05665639 ... 0.03892663 0.756121   0.02340105]
 [0.01350205 0.         0.04320683 ... 0.02599499 0.742682   0.0190739 ]
 [0.05665639 0.04320683 0.         ... 0.01968244 0.70020361 0.04890605]
 ...
 [0.03892663 0.02599499 0.01968244 ... 0.         0.71852238 0.03639647]
 [0.756121   0.742682   0.70020361 ... 0.71852238 0.         0.74

Additional things that can be done: Use list or similar data structure to append current index_list to it every time clustering is performed, to be able to see the clustering history